In [2]:
import google.generativeai as genai
import textwrap
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("API_KEY")
nlpfpath='Intern NLP Dataset.xlsx'

# print(API_KEY)
# gptkey=gpt secret key
# genai.configure(api_key=API_KEY)

sk-proj-Lz14RJFfkIvZCCrKfBTTT3BlbkFJtesTimKB6zJlNBAXWuK9


In [2]:
df=pd.read_excel(nlpfpath)

In [3]:
columns=df.columns.tolist()
columns

['Time',
 'Is Male',
 'Is Female',
 'Is Hijab',
 'Is Child',
 'Is Niqab',
 'Has Bag',
 'Cluster ID']

In [4]:


df=pd.read_excel(nlpfpath)
df.head()
#  convert this file to sql file
import sqlite3
conn = sqlite3.connect('nlp.db')
c = conn.cursor()
df.to_sql('nlp', conn, if_exists='replace', index = False)
conn.commit()



In [5]:
print (columns)

['Time', 'Is Male', 'Is Female', 'Is Hijab', 'Is Child', 'Is Niqab', 'Has Bag', 'Cluster ID']


In [20]:


prompt = f"""
You are an expert in converting English questions to SQL query!
The SQL database has the name nlp and has the following columns 
- {columns}

For example,
Example 1 - How many entries of records are present?, 
the SQL command will be something like this: SELECT COUNT(*) FROM nlp;

also the SQL code should not have ``` 
in the beginning or end and SQL word in output
"""


print(prompt)
def read_sql_query(sql,db):
    conn=sqlite3.connect(db)
    cur=conn.cursor()
    cur.execute(sql)
    rows=cur.fetchall()
    conn.commit()
    conn.close()
    for row in rows:
        pass
        print(row)
    return rows
def get_gemini_response(question,prompt):
    model=genai.GenerativeModel('gemini-pro')
    response=model.generate_content([prompt,question])
    return response.text
# print(query)
while True:
    try:
        # question='Which minute did I get the most visitors, and Who is my most common visitor?'
        question= 'and Who is my most common visitor?'
        query=get_gemini_response(question,prompt)
        print(query)
        result=read_sql_query(query,'nlp.db')
        # print(result)
        break

    except Exception as e:
        print(e)
        ##add the error to the prompt[0]
        print
        prompt[0]=prompt[0]+"repeat the process to fix this error"+ "here is the query: " +query+"here is the error: " +str(e)



You are an expert in converting English questions to SQL query!
The SQL database has the name nlp and has the following columns 
- ['Time', 'Is Male', 'Is Female', 'Is Hijab', 'Is Child', 'Is Niqab', 'Has Bag', 'Cluster ID']

For example,
Example 1 - How many entries of records are present?, 
the SQL command will be something like this: SELECT COUNT(*) FROM nlp;

also the SQL code should not have ``` 
in the beginning or end and SQL word in output

SELECT "Cluster ID", COUNT(*) AS "Frequency"
FROM nlp
GROUP BY "Cluster ID"
ORDER BY "Frequency" DESC
LIMIT 1;
(36, 7)


In [ ]:
select = genai.protos.Schema(
        type=genai.protos.Type.STRING,
    )
selectcols = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=select
)
where = genai.protos.Schema(
        type=genai.protos.Type.STRING,
    )
wherecols = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=where
)
group = genai.protos.Schema(
        type=genai.protos.Type.STRING,
    )
groupcols = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=group
)
order = genai.protos.Schema(
        type=genai.protos.Type.STRING,
    )
ordercols = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=order
)
queryobj = genai.protos.FunctionDeclaration(
name="structure_query",
description=textwrap.dedent("""\
    structure a query from the given text
    """),
parameters=genai.protos.Schema(
    type=genai.protos.Type.OBJECT,
    properties = {
        'selectcols': selectcols,
        'wherecols':wherecols,
        'groupcols':groupcols,
        'ordercols':ordercols,
        
        
    }
)
)

In [ ]:
print(queryobj)

name: "structure_query"
description: "structure a query from the given text\n"
parameters {
  type_: OBJECT
  properties {
    key: "wherecols"
    value {
      type_: ARRAY
      items {
        type_: STRING
      }
    }
  }
  properties {
    key: "selectcols"
    value {
      type_: ARRAY
      items {
        type_: STRING
      }
    }
  }
  properties {
    key: "ordercols"
    value {
      type_: ARRAY
      items {
        type_: STRING
      }
    }
  }
  properties {
    key: "groupcols"
    value {
      type_: ARRAY
      items {
        type_: STRING
      }
    }
  }
}



In [ ]:
model = model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    tools = [queryobj])

['Time',
 'Is Male',
 'Is Female',
 'Is Hijab',
 'Is Child',
 'Is Niqab',
 'Has Bag',
 'Cluster ID']

In [ ]:
#get the data types
data_types = df.dtypes
data_types

Time          datetime64[ns]
Is Male                int64
Is Female              int64
Is Hijab               int64
Is Child               int64
Is Niqab               int64
Has Bag                int64
Cluster ID             int64
dtype: object

In [ ]:
query='Which minute did I get the most visitors, and Who is my most common visitor?'
modelinput=f"""
Please structure this query: {query} on these columns: {columns} where the values are binary except for time and cluster ID
"""
print(modelinput)
result = model.generate_content(
    modelinput
,tool_config={'function_calling_config':'ANY'})    
fc=result.candidates[0].content.parts[0].function_call
fc=type(fc).to_dict(fc)
fc


Please structure this query: Which minute did I get the most visitors, and Who is my most common visitor? on these columns: ['Time', 'Is Male', 'Is Female', 'Is Hijab', 'Is Child', 'Is Niqab', 'Has Bag', 'Cluster ID'] where the values are binary except for time and cluster ID



{'name': 'structure_query',
 'args': {'selectcols': ['Time', 'count(*)'],
  'wherecols': ['Is Visitor  =  1'],
  'groupcols': ['Time'],
  'ordercols': ['count(*) DESC']}}

In [ ]:
# df.query(fc['args']['selectcols'][0])

KeyError: "None of [DatetimeIndex(['2024-07-17 14:30:38.456000', '2024-07-17 14:31:41.128000',\n               '2024-07-17 14:33:14.869000', '2024-07-17 14:33:16.417000',\n               '2024-07-17 14:33:18.821000', '2024-07-17 14:33:21.427000',\n               '2024-07-17 14:34:04.205000', '2024-07-17 14:36:01.130000',\n               '2024-07-17 14:36:40.263000', '2024-07-17 14:37:37.979000',\n               '2024-07-17 14:37:40.097000', '2024-07-17 14:40:41.511000',\n               '2024-07-17 14:42:56.267000', '2024-07-17 14:42:56.521000',\n               '2024-07-17 14:43:37.136000', '2024-07-17 14:44:17.314000',\n               '2024-07-17 14:45:07.718000', '2024-07-17 14:45:16.084000',\n               '2024-07-17 14:45:17.967000', '2024-07-17 14:46:05.075000',\n               '2024-07-17 14:46:06.585000', '2024-07-17 14:46:08.832000',\n               '2024-07-17 14:49:14.912000', '2024-07-17 14:50:55.856000',\n               '2024-07-17 14:50:56.117000', '2024-07-17 14:51:13.52